In [ ]:
!pip install sentinelsat
!pip install ipywidgets
!pip install --upgrade attrs
!pip install h5netcdf
!pip install netCDF4
!pip install scipy
!pip uninstall xarray -y
!pip install xarray
!pip install matplotlib
!pip install keras
!pip install tensorflow
!pip install tensorflow-gpu
!pip install -q -U keras-tuner
!conda install -c conda-forge harp -y

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import ipywidgets
import numpy as np
from glob import iglob

import xarray as xr
import numpy as np
from os.path import join
from matplotlib import pyplot as plt
import os
from datetime import date, timedelta

In [ ]:
import numpy as np
import regex as re
from glob import iglob
from os.path import join
import os

no_of_samples = 1529
channels = 2
image_dim = 16


input_length = 10
output_length = 5

product_path_NO2 = "Dataset_NO2_Processed/"
product_path_LST = "Dataset_LST_Processed/"

input_files_NO2 = sorted(list(iglob(join(product_path_NO2, '*NO2*.npy'), recursive=False)))
input_files_LST = sorted(list(iglob(join(product_path_LST, '*LST*.npy'), recursive=False)))

no_of_samples = len(input_files_NO2)
no_train_samples = int(no_of_samples*0.8)
no_test_samples = no_of_samples - no_train_samples
print("Number of samples: ", no_of_samples)

train_samples = np.zeros(shape=(no_train_samples,image_dim,image_dim,channels))
test_samples = np.zeros(shape=(no_test_samples,image_dim,image_dim,channels))
count = 0

#Stack the NO2 and LST data in separate channels of the same frame.
for NO2 in input_files_NO2:
    image_no2 = np.load(NO2)
    #21-29 for new 15-23 for old
    image_LST = np.load(join(product_path_LST, NO2[(len(product_path_NO2)+21):(len(product_path_NO2)+29)]) + "_LST.npy")
    
    if image_no2.shape == (1,image_dim,image_dim):
        image_no2 = image_no2.reshape(image_dim,image_dim,1)
        image_LST = image_LST.reshape(image_dim,image_dim,1)
        
        if count < no_train_samples:
            train_samples[count] = (np.concatenate((image_no2, image_LST), axis=2))
        else:
            test_samples[count-no_train_samples] = (np.concatenate((image_no2, image_LST), axis=2))
        
        
    count = count + 1
    
train_samples.clip(min=0, out=train_samples)
test_samples.clip(min=0, out=test_samples)

# Computing statistics for normalisation.
min = np.nanmin(train_samples, axis = (0,1,2))
max = np.nanmax(train_samples, axis = (0,1,2))
mean = np.nanmean(train_samples, axis = (0,1,2))
std = np.nanstd(train_samples, axis = (0,1,2))

train_samples = train_samples - min
train_samples = train_samples/(max-min)
test_samples = test_samples - min
test_samples = test_samples/(max-min)

train_samples = np.nan_to_num(train_samples)
test_samples = np.nan_to_num(test_samples)

train_samples = np.clip(train_samples, 0, 1, out=train_samples)
test_samples = np.clip(test_samples, 0, 1, out=test_samples)

In [ ]:
#Plot for Figure 4.3
#plt.hist(train_samples[:,:,:,0].reshape(len(train_samples),-1))
#plt.plot()

In [ ]:
#4D tensor to 5D tensor
from sklearn.utils import shuffle

count = 0

# Create model inputs and corresponding labels using bundles of sequential data.
# The following code in this cell has been adapted from the work of Muthukumar et. al.
train_bundles = np.empty((no_train_samples-input_length,input_length,image_dim,image_dim,channels))
for i in range(no_train_samples-input_length):
    bundle = np.array([train_samples[i + k] for k in range(input_length)])
    train_bundles[i] = bundle
no_train_samples = train_bundles.shape[0]
    
test_bundles = np.empty((no_test_samples-input_length,input_length,image_dim,image_dim,channels))
for i in range(no_test_samples-input_length):
    bundle = np.array([test_samples[i + k] for k in range(input_length)])
    test_bundles[i] = bundle
no_test_samples = test_bundles.shape[0]

X_train = train_bundles[:no_train_samples-(input_length)]
X_val = test_bundles[:int(no_test_samples*0.5)]
X_test = test_bundles[int(no_test_samples*0.5)+input_length:no_test_samples-(input_length)]

y_train = np.expand_dims(train_bundles[input_length:,:output_length,:,:,0],-1)
y_val = np.expand_dims(test_bundles[input_length:int(no_test_samples*0.5)+input_length,:output_length,:,:,0],-1)
y_test = np.expand_dims(test_bundles[int(no_test_samples*0.5)+2*input_length:,:output_length,:,:,0],-1)


X_train, y_train = shuffle(X_train, y_train, random_state=7)

print(X_train.shape, "X_train shape")
print(X_val.shape, "X_val shape")
print(X_test.shape, "X_test shape")

print(y_train.shape, "y_train shape")
print(y_val.shape, "y_val shape")
print(y_test.shape, "y_test shape")

In [ ]:
!pip install keras
!pip install tensorflow
!pip install tensorflow-gpu
import tensorflow as tf
from tensorflow.keras.models import Sequential
import keras
from keras.models import Model
from keras.layers.convolutional import Conv2D
from keras.layers import ConvLSTM2D, Input
from keras.layers import RepeatVector
from keras.layers import BatchNormalization

from keras.backend import expand_dims, repeat_elements
from keras.layers import Lambda

import keras.backend as K


# Implementation of the shrinkage loss function.
def shrinkage_loss(y_true, y_pred):
 
    error = tf.subtract(y_pred, y_true)
    squared_error = K.square(error) 
    
    a = 10.0
    c = 0.1
    
    shrinkage_loss = (squared_error/(1 + K.exp(a * (c - error))))
                
    loss = tf.reduce_mean(shrinkage_loss, axis=(1,2,3,4))
    
    return loss

# Implementation of the shrinkage version of the mean absolute loss.
def shrinkage_absolute_loss(y_true, y_pred):
 
    error = y_pred - y_true
    squared_error = K.abs(error) 
    
    a = 10.0
    c = 0.3
    
    shrinkage_loss = squared_error/(1 + K.exp(a * (c - error)))
    
    loss = K.mean(shrinkage_loss, axis=(1,2,3,4))
    
    return loss

def repeatV(x):
    return repeat_elements(expand_dims(x, axis=1), output_length, 1)

# Implementation of the Convolutional LSTM model.
inputs = Input(shape=(input_length, image_dim, image_dim, channels))
enc_layer_1 = ConvLSTM2D(filters=32, kernel_size=7,
                   input_shape=(input_length, image_dim, image_dim, channels),
                   padding='same', return_sequences=True, data_format='channels_last')(inputs)
enc_batch_norm_1 = BatchNormalization()(enc_layer_1)

enc_layer_2 = ConvLSTM2D(filters=64, kernel_size=7,
                   padding='same', return_sequences=True, data_format='channels_last')(enc_batch_norm_1)
enc_batch_norm_2 = BatchNormalization()(enc_layer_2)

enc_layer_3 = ConvLSTM2D(filters=128, kernel_size=9,
                   padding='same', return_sequences=True, data_format='channels_last')(enc_batch_norm_2)
enc_batch_norm_3 = BatchNormalization()(enc_layer_3)

enc_layer_4 = ConvLSTM2D(filters=128, kernel_size=9,
                   padding='same', return_sequences=False, data_format='channels_last')(enc_batch_norm_3)
enc_batch_norm_4 = BatchNormalization()(enc_layer_4)

repeat = Lambda(repeatV)(enc_batch_norm_4)

dec_layer_1 = ConvLSTM2D(filters=128, kernel_size=11,
                   input_shape=(output_length, image_dim, image_dim, channels),
                   padding='same', return_sequences=True, data_format='channels_last')(repeat)
dec_batch_norm_1 = BatchNormalization()(dec_layer_1)

dec_layer_2 = ConvLSTM2D(filters=128, kernel_size=9,
                   padding='same', return_sequences=True, data_format='channels_last')(dec_batch_norm_1)
dec_batch_norm_2 = BatchNormalization()(dec_layer_2)

dec_layer_3 = ConvLSTM2D(filters=64, kernel_size=9,
                   padding='same', return_sequences=True, data_format='channels_last')(dec_batch_norm_2)
dec_batch_norm_3 = BatchNormalization()(dec_layer_3)

dec_layer_4 = ConvLSTM2D(filters=32, kernel_size=7,
                   padding='same', return_sequences=True, data_format='channels_last')(dec_batch_norm_3)
dec_batch_norm_4 = BatchNormalization()(dec_layer_4)

y = Conv2D(filters=1, kernel_size=9,
               activation='sigmoid',
               padding='same', data_format='channels_last')(dec_batch_norm_4)

seq = Model(inputs, y)

# Different loss functions, e.g. shrinkage_loss, "mean_squared_error", etc., can be used to compile the model.
seq.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=0.0005, beta_1=0.90, beta_2=0.999, amsgrad=False), metrics=["mean_absolute_error"])

In [ ]:
# The following code was used to set up the hyperparameter tuner.

# !pip install keras
# !pip install tensorflow
# !pip install tensorflow-gpu
# !pip install -q -U keras-tuner
# import keras_tuner as kt
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# import keras
# from keras.models import Model
# from keras.layers.convolutional import Conv2D
# from keras.layers import ConvLSTM2D, Input
# from keras.layers import RepeatVector
# from keras.layers import BatchNormalization

# from keras.backend import expand_dims, repeat_elements
# from keras.layers import Lambda

# import keras.backend as K

# def shrinkage_loss(y_true, y_pred):
 
#     error = y_pred - y_true
#     squared_error = K.square(y_pred - y_true)
    
#     a = 10.0
#     c = 0.1
    
#     shrinkage_loss = squared_error/(1 + K.exp(a * (c - error)))
                
#     loss = K.mean(shrinkage_loss, axis=(1,2,3,4))
    
#     return loss

# def repeatV(x):
#     return repeat_elements(expand_dims(x, axis=1), output_length, 1)

# def model_builder_enc_dec(hp):
    
#     inputs = Input(shape=(input_length, image_dim, image_dim, channels))
#     layer_input = inputs
#     num_enc_layers = hp.Int('enc_layers', 1, 8)
#     num_dec_layers = hp.Int('dec_layers', 1, 8)
#     for i in range(num_enc_layers):
#         if(i == num_enc_layers-1):
#             return_sequence = False
#         else:
#             return_sequence = True
    
#         layer = ConvLSTM2D(filters=hp.Choice(f'filters_enc_{i}', [8,16,32,64,128]), kernel_size=hp.Int(f'kernel_enc_{i}', min_value=3, max_value=11, step=2),
#                            input_shape=(input_length, image_dim, image_dim, channels),
#                            padding='same', return_sequences=return_sequence, data_format='channels_last')(layer_input)
#         batch_norm = BatchNormalization()(layer)
        
#         layer_input = batch_norm

#     repeat = Lambda(repeatV)(layer_input)
    
#     layer_input = repeat
    
#     for i in range(num_enc_layers):
#         layer = ConvLSTM2D(filters=hp.Choice(f'filters_dec_{i}', [8,16,32,64,128]), kernel_size=hp.Int(f'kernel_dec_{i}', min_value=3, max_value=11, step=2),
#                            input_shape=(input_length, image_dim, image_dim, channels),
#                            padding='same', return_sequences=True, data_format='channels_last')(layer_input)
#         batch_norm = BatchNormalization()(layer)
        
#         layer_input = batch_norm

#     kernel = hp.Int('kernel', min_value=3, max_value=11, step=2)
        
#     y = Conv2D(filters=1, kernel_size=kernel,
#                    activation='sigmoid',
#                    padding='same', data_format='channels_last')(layer_input)
    
#     seq = Model(inputs, y)
    
#     learning_rate = hp.Choice('learning_rate', [0.005, 0.001, 0.0005, 0.0001, 0.00005])
#     beta_1 = hp.Choice('beta_1', [0.7, 0.8, 0.9, 0.999])
#     beta_2 = hp.Choice('beta_2', [0.7, 0.8, 0.9, 0.999])
    
#     seq.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2, amsgrad=False), metrics=["mean_absolute_error"])
    
#     return seq
    

In [ ]:
# The following code was used to run the hyperparameter tuner.

# tuner = kt.Hyperband(model_builder_enc_dec,
#                      objective=kt.Objective("val_mean_absolute_error", direction="min"),
#                      max_epochs=40,
#                      factor=3,
#                      directory='tuner',
#                      project_name='ConvLSTM_Tune')

# tuner.search(X_train, y_train, epochs=30, validation_data=(X_val, y_val))
# best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]


In [ ]:
# Load saved model.
# Choose model from:
# 'saved_models/model_mse'
# 'saved_models/model_shrinkage'
# 'saved_models/model_wide'

model_path = 'saved_models/model_shrinkage'

json_file = open(model_path + '.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
seq = keras.models.model_from_json(loaded_model_json)
# load weights into new model
seq.load_weights(model_path + ".h5")
print("Model Loaded")
seq.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, amsgrad=False), metrics=["mean_absolute_error"])

# Use following code to train model from scratch.
# seq.summary()
# history = seq.fit(X_train, y_train, batch_size=32, epochs=30, validation_data=(X_val, y_val))
# Y_hat = seq.predict(X_test, verbose=0)
# print(Y_hat.shape)

In [ ]:
# Code for saving model and weights.
# model_json = seq.to_json()
# with open("model_LST.json", "w") as json_file:
#     json_file.write(model_json)
    
# # Save weights to HDF5
# seq.save_weights("model_LST.h5")
# print("Saved model to disk")

In [ ]:
# The following visualization code in this cell is taken from Muthukumar et. al's work.
# Randomly visualizes frame sequences from the test set.
def visualization(y, y_hat, n_slots):
    
    import random
    import matplotlib.pyplot as plt
    import numpy as np
    length = y_hat.shape[0]
    random_ind = [x for x in range(0,n_slots)]
    setCount = 1;
    for ind in random_ind:
        setCount = setCount+1;
        fig, (ax1, ax2)= plt.subplots(1, 2, sharey=True, sharex=True, figsize=(10, 5))

        y_pre = y_hat[ind,0,:,:,0]
        ax1.imshow(y_pre)

        y_truth = y[ind,0,:,:,0]
        ax2.imshow(y_truth)

        plt.text(2, -0.8, 'Ground truth Frame 1', fontsize=13)
    
        fig, (ax1, ax2)= plt.subplots(1, 2, sharey=True, sharex=True, figsize=(10, 5))
        
        y_pre = y_hat[ind,1,:,:,0]
        ax1.imshow(y_pre)
        
        y_truth = y[ind,1,:,:,0]
        ax2.imshow(y_truth)

        plt.text(2, -0.8, 'Ground truth Frame 2', fontsize=13)
    
    
        fig, (ax1, ax2)= plt.subplots(1, 2, sharey=True, sharex=True, figsize=(10, 5))
        
        y_pre = y_hat[ind,2,:,:,0]
        ax1.imshow(y_pre)

        y_truth = y[ind,2,:,:,0]
        ax2.imshow(y_truth)

        plt.text(2, -0.8, 'Ground truth Frame 3', fontsize=13)
    
        fig, (ax1, ax2)= plt.subplots(1, 2, sharey=True, sharex=True, figsize=(10, 5))

        y_pre = y_hat[ind,3,:,:,0]
        ax1.imshow(y_pre)

        y_truth = y[ind,3,:,:,0]
        ax2.imshow(y_truth)

        plt.text(2, -0.8, 'Ground truth Frame 4', fontsize=13)
    
        fig, (ax1, ax2)= plt.subplots(1, 2, sharey=True, sharex=True, figsize=(10, 5))

        y_pre = y_hat[ind,4,:,:,0]
        ax1.imshow(y_pre)

        y_truth = y[ind,4,:,:,0]
        ax2.imshow(y_truth)

        plt.text(2, -0.8, 'Ground truth Frame 5', fontsize=13)

In [ ]:
visualization(y_test, Y_hat, 5)

In [ ]:
!pip install scikit-image
import skimage
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

# Calculate raw normalised statistics for evaluation.

Y_hat = seq.predict(X_test, verbose=0)

img_dim = 16
resi = []
hati = []
tesi = []

count = 0
for j in range(0, 5, 2):
    lst = []
    lstres = []
    lstresidual = []
    hat = []
    test = []
    for ind in range(Y_hat.shape[0]):

        img_hat = Y_hat[ind][j].reshape(img_dim,img_dim).astype('float64')
        img_test = y_test[ind][j].reshape(img_dim,img_dim)

        mae = mean_absolute_error(img_hat, img_test)
        res = np.mean(img_hat-img_test)
        residual = img_hat-img_test
        
        lstresidual.append(residual)
        lst.append(mae)
        lstres.append(res)
        hat.append(img_hat)
        test.append(img_test)
    
    resi.append(lstresidual)
    hati.append(hat)
    tesi.append(test)
    
    lst = np.array(lst)
    mean = np.mean(lst)
    std = np.std(lst)
    meansim = np.mean(lstsim)
    stdsim = np.std(lstsim)
    meanres = np.mean(lstres)
    stdres = np.std(lstres)
    
    print("mean", mean)
    print('---')

In [ ]:
# Plot temporal average residual errors.

avg = np.mean(resi, axis=1)

f, axarr = plt.subplots(1,len(avg))
for frame_ind in range(len(avg)):
    
    data_mean = 119
    b = axarr[frame_ind].imshow(avg[frame_ind]*data_mean, interpolation='none')
    b.axes.get_xaxis().set_visible(False)
    b.axes.get_yaxis().set_visible(False)
    axarr[frame_ind].title.set_text('Day ' + str(2*(frame_ind + 1)-1))
   

cbar = f.colorbar(b, ax=axarr.ravel().tolist(), orientation="horizontal")
cbar.set_label('Mean Error (Petamolecules/cm2)')
plt.show()